In [1]:
from investpy.stocks import get_stock_historical_data as gshd
from datetime import datetime, timedelta
import pandas as pd
import os

In [2]:
kospi_route = '.\\kospi\\'
datas = os.listdir(kospi_route)

In [3]:
dfs, names = {}, []
for data in datas:
    csv_route = kospi_route + data
    csv_name = data.split('-')
    if len(csv_name) > 2:
        csv_name = '-'.join(csv_name[:2])
    else:
        csv_name = csv_name[0]
    
    df = pd.read_csv(csv_route, encoding='utf-8', sep=',')
    dfs[csv_name] = df
    names.append(csv_name)

In [4]:
articles = pd.read_csv('.\\articles_ko.csv', encoding='utf-8', sep=',')
co_articles = []
for name in names:
    sep_articles = articles[articles['target'] == name]
    co_articles.append(sep_articles)

In [7]:
updown = {}
for co in co_articles:
    co_dates = list(co['date'])
    updown[list(co['target'].drop_duplicates())[0]] = co_dates

In [8]:
data = [[], []]
columns = ['date', 'updown']
for i in range(len(dfs)):
    kospi, target = sorted(dfs.keys())[i], sorted(updown.keys())[i]
    money, dates = dfs[kospi], updown[target]
    kospi_dates = list(money.Date.drop_duplicates())

    check, index = [], []
    for date in dates:
        if date in kospi_dates:
            idx = kospi_dates.index(date)
            if idx != len(kospi_dates) - 1:
                now = money.iloc[idx, :]
                to = money.iloc[idx + 1, :]
                diff = to.Close - now.Close
                if diff > 0:
                    check.append(True)
                elif diff == 0:
                    check.append(None)
                else:
                    check.append(False)
                index.append(date)
            else:
                check.append(None)
                index.append(date)
        else:
            for compare in kospi_dates:
                real_compare = datetime.strptime(compare, '%Y-%m-%d')
                real_date = datetime.strptime(date, '%Y-%m-%d')
                if abs(real_compare - real_date) <= timedelta(days=1):
                    idx = kospi_dates.index(compare)
                    break
            if idx != 0:
                now = money.iloc[idx - 1, :]
                to = money.iloc[idx, :]
                diff = to.Close - now.Close
                if diff > 0:
                    check.append(True)
                elif diff == 0:
                    check.append(None)
                else:
                    check.append(False)
                index.append(date)
            else:
                check.append(None)
                index.append(date)
    data[0] += index
    data[1] += check

In [9]:
new_df = pd.DataFrame(data).T
new_df.columns = columns

In [10]:
new_df.set_index('date', inplace=True)
new_df.to_csv('.\\label.csv', encoding='utf-8', sep=',')